In [1]:
import torch

In [11]:
"""Compare speed of different models with batch size 16"""
import torch
from torchvision.models import resnet, densenet, vgg, squeezenet,inception
from torch.autograd import Variable
import torch.nn as nn
import time
import argparse
import os
import numpy as np
import pandas as pd
import pandas
import glob
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple

def print_num_gpus():
    print('Number of GPUs on current device %i' % torch.cuda.device_count())


def print_cuda_version():
    print('CUDA version = %s' % torch.version.cuda)


def print_cudnn_version():
    print('cudnn version=', torch.backends.cudnn.version())

def print_device_name():
    print('device_name=', torch.cuda.get_device_name(0))

def arr_train():
    result_path=os.path.join(os.getcwd(),'results')
    arr_train=[i for i in glob.glob(result_path+'/*training*.csv')]
    arr_train.sort()
    return arr_train

def arr_inference():
    result_path=os.path.join(os.getcwd(),'results')
    arr_inference=[i for i in glob.glob(result_path+'/*inference*.csv')]
    arr_inference.sort()
    return arr_inference


def total_model(arr,device_name):

    model_name=arr[0].split('/')[-1].split('_')[0]
    print(model_name)
    type=arr[0].split('/')[-1].split('_')[3]
    n_groups = 13

    half=pd.read_csv(arr[0])
    single=pd.read_csv(arr[1])


    means_half =half.mean().values
    std_half =half.std().values

    means_single =single.mean().values
    std_single =single.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.5

    opacity = 1.0
    error_config = {'ecolor': '0.3'}



    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_half, error_kw=error_config,
                    label='half')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_single, error_kw=error_config,
                    label='single')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title("total_"+type+"_"+model_name)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(single.columns,rotation=60, fontsize=9)

    ax.legend()

    fig.tight_layout()
    plt.savefig(device_name+'total.png',dpi=400)



def model_plot(arr,model):

    model_name=arr[0].split('/')[-1].split('_')[0]
    
    type=arr[0].split('/')[-1].split('_')[3]
    double=pd.read_csv(arr[0])
    half=pd.read_csv(arr[1])
    single=pd.read_csv(arr[2])
    if model.lower() =='densenet':
        n_groups = 4
        double=double[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        half=half[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        single=single[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
    elif model.lower() =='resnet':
        n_groups = 5
        double=double[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        half=half[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        single=single[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
    elif model.lower() =='squeezenet':
        n_groups = 2
        double=double[['squeezenet1_0','squeezenet1_1']]
        half=half[['squeezenet1_0','squeezenet1_1']]
        single=single[['squeezenet1_0','squeezenet1_1']]
    elif model.lower() =='vgg':
        n_groups = 4
        double=double[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        half=half[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        single=single[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
    else:
        raise NotImplementedError("To be implemented")


    means_double =double.mean().values
    std_double =double.std().values

    means_half =half.mean().values
    std_half =half.std().values

    means_single =single.mean().values
    std_single =single.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.25

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = ax.bar(index, means_double, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_double, error_kw=error_config,
                    label='double')

    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_half, error_kw=error_config,
                    label='half')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='g',
                    yerr=std_single, error_kw=error_config,
                    label='single')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title(model+'_'+type+"_"+model_name)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(double.columns,rotation=45, fontsize=9)

    ax.legend()
    fig.tight_layout()
    plt.savefig(model+'.png',dpi=300)


def arr_type(arr,type):
    arr=[x for x in arr if not '(2)' in x ]
    if type == 'single':
        temp=[x for x in arr if 'single' in x ]
    elif type == 'double':
        temp=[x for x in arr if 'double' in x ]
    elif type == 'half':
        temp=[x for x in arr if 'half' in x ]
    return temp

def model_plot2(arr,model):
    #model_name=arr[0].split('/')[-1].split('_')[0]
    type=arr[0].split('/')[-1].split('_')[3]
    ti1080=pd.read_csv(arr[0])
    ti2080=pd.read_csv(arr[1])
    titan=pd.read_csv(arr[2])
    if model.lower() =='densenet':
        n_groups = 4
        ti1080=ti1080[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        ti2080=ti2080[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        titan=titan[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
    elif model.lower() =='resnet':
        n_groups = 5
        ti1080=ti1080[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        ti2080=ti2080[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        titan=titan[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
    elif model.lower() =='squeezenet':
        n_groups = 2
        ti1080=ti1080[['squeezenet1_0','squeezenet1_1']]
        ti2080=ti2080[['squeezenet1_0','squeezenet1_1']]
        titan=titan[['squeezenet1_0','squeezenet1_1']]
    elif model.lower() =='vgg':
        n_groups = 4
        ti1080=ti1080[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        ti2080=ti2080[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        titan=titan[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
    else:
        raise NotImplementedError("To be implemented")


    means_double =ti1080.mean().values
    std_double =ti1080.std().values

    means_half =ti2080.mean().values
    std_half =ti2080.std().values

    means_single =titan.mean().values
    std_single =titan.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.25

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = ax.bar(index, means_double, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_double, error_kw=error_config,
                    label='1080ti')

    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_half, error_kw=error_config,
                    label='2080ti')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='g',
                    yerr=std_single, error_kw=error_config,
                    label='TitanV')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title(model+'_'+type)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(ti1080.columns,rotation=45, fontsize=9)

    ax.legend()
    fig.tight_layout()
    plt.savefig(model+'_'+type+'_'+model_name+'.png',dpi=300)

def print_info():
    print_num_gpus()
    print_cuda_version()
    print_cudnn_version()
    print_device_name()

print_info()

MODEL_LIST = {
    resnet:['resnet50'],
    densenet: ['densenet121'],
    squeezenet: ['squeezenet1_0'],
    vgg: ['vgg16']

}

precision=['single','half']
device_name=torch.cuda.get_device_name(0)

# Training settings
parser = argparse.ArgumentParser(description='PyTorch Benchmarking')
parser.add_argument('--WARM_UP','-w', type=int,default=5, required=False, help="Num of warm up")
parser.add_argument('--NUM_TEST','-n', type=int,default=50,required=False, help="Num of Test")
parser.add_argument('--BATCH_SIZE','-b', type=int, default=20, required=False, help='Num of batch size')
parser.add_argument('--NUM_CLASSES','-c', type=int, default=1000, required=False, help='Num of class')
parser.add_argument('--NUM_GPU','-g', type=int, default=1, required=False, help='Num of class')
args = parser.parse_args(['-w 5','-n 50','-b 20','-c 1000','-c 1'])
device_name+='_'+str(args.NUM_GPU)+'_gpus_'
args.BATCH_SIZE*=args.NUM_GPU
torch.backends.cudnn.benchmark = True
def train(type='single'):
    """use fake image for training speed test"""
    img = Variable(torch.randn(args.BATCH_SIZE, 3, 224, 224)).cuda()
    target = Variable(torch.LongTensor(args.BATCH_SIZE).random_(args.NUM_CLASSES)).cuda()
    criterion = nn.CrossEntropyLoss()
    benchmark = {}
    for model_type in MODEL_LIST.keys():
        for model_name in MODEL_LIST[model_type]:
            model = getattr(model_type, model_name)(pretrained=False)
            if args.NUM_GPU > 1:
                model = nn.DataParallel(model)
            if type is 'double':
                model=model.double()
                img=img.double()
            elif type is 'single':
                model=model.float()
                img=img.float()
            elif type is 'half':
                model=model.half()
                img=img.half()
            model.cuda()
            model.train()
            durations = []
            print('Benchmarking Training '+type+' precision type %s' % (model_name))
            for step in range(args.WARM_UP + args.NUM_TEST):
                torch.cuda.synchronize()
                start = time.time()
                model.zero_grad()
                prediction = model.forward(img)
                loss = criterion(prediction, target)
                loss.backward()
                torch.cuda.synchronize()
                end = time.time()
                if step >= args.WARM_UP:
                    durations.append((end - start)*1000)
            del model
            benchmark[model_name] = durations
    return benchmark

def inference(type='single'):
    benchmark = {}
    img = Variable(torch.randn(args.BATCH_SIZE, 3, 224, 224), requires_grad=True).cuda()
    with torch.no_grad():
        for model_type in MODEL_LIST.keys():
            for model_name in MODEL_LIST[model_type]:
                model = getattr(model_type, model_name)(pretrained=False)
                if args.NUM_GPU > 1:
                    model = nn.DataParallel(model)
                if type is 'double':
                    model=model.double()
                    img=img.double()
                elif type is 'single':
                    model=model.float()
                    img=img.float()
                elif type is 'half':
                    model=model.half()
                    img=img.half()
                model.cuda()
                model.eval()
                durations = []
                print('Benchmarking Inference '+type+' precision type %s ' % (model_name))
                for step in range(args.WARM_UP + args.NUM_TEST):
                    torch.cuda.synchronize()
                    start = time.time()
                    model.forward(img)
                    torch.cuda.synchronize()
                    end = time.time()
                    if step >= args.WARM_UP:
                        durations.append((end - start)*1000)
                del model
                benchmark[model_name] = durations
    return benchmark



if __name__ == '__main__':
    os.makedirs('results', exist_ok=True)
    for i in precision:
        torch.cuda.empty_cache()
        training_benchmark = pandas.DataFrame(train(i))
        training_benchmark.to_csv('results/'+device_name+"_"+i+'_model_training_benchmark.csv', index=False)
        inference_benchmark = pandas.DataFrame(inference(i))
        inference_benchmark.to_csv('results/'+device_name+"_"+i+'_model_inference_benchmark.csv', index=False)
    
    train=arr_train()
    inference=arr_inference()

    

Number of GPUs on current device 1
CUDA version = 10.0
cudnn version= 7401
device_name= GeForce GTX 1060 6GB
Benchmarking Training single precision type resnet50
Benchmarking Training single precision type densenet121
Benchmarking Training single precision type squeezenet1_0
Benchmarking Training single precision type vgg16
Benchmarking Inference single precision type resnet50 
Benchmarking Inference single precision type densenet121 
Benchmarking Inference single precision type squeezenet1_0 
Benchmarking Inference single precision type vgg16 
Benchmarking Training half precision type resnet50
Benchmarking Training half precision type densenet121
Benchmarking Training half precision type squeezenet1_0
Benchmarking Training half precision type vgg16
Benchmarking Inference half precision type resnet50 
Benchmarking Inference half precision type densenet121 
Benchmarking Inference half precision type squeezenet1_0 
Benchmarking Inference half precision type vgg16 


In [1]:
import torch


def print_num_gpus():
    print('Number of GPUs on current device %i' % torch.cuda.device_count())


def print_cuda_version():
    print('CUDA version = %s' % torch.version.cuda)


def print_cudnn_version():
    print('cudnn version=', torch.backends.cudnn.version())

def print_device_name():
    print('device_name=', torch.cuda.get_device_name(0))
    

def print_info():
    print_num_gpus()
    print_cuda_version()
    print_cudnn_version()
    print_device_name()

In [6]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple


def arr_train():
    result_path=os.path.join(os.getcwd(),'results')
    arr_train=[i for i in glob.glob(result_path+'/*training*.csv')]
    arr_train.sort()
    return arr_train

def arr_inference():
    result_path=os.path.join(os.getcwd(),'results')
    arr_inference=[i for i in glob.glob(result_path+'/*inference*.csv')]
    arr_inference.sort()
    return arr_inference


def total_model(arr,device_name):

    model_name=arr[0].split('/')[-1].split('_')[0]
    print(model_name)
    type=arr[0].split('/')[-1].split('_')[3]
    n_groups = 13

    half=pd.read_csv(arr[0])
    single=pd.read_csv(arr[1])


    means_half =half.mean().values
    std_half =half.std().values

    means_single =single.mean().values
    std_single =single.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.5

    opacity = 1.0
    error_config = {'ecolor': '0.3'}



    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_half, error_kw=error_config,
                    label='half')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_single, error_kw=error_config,
                    label='single')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title("total_"+type+"_"+model_name)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(single.columns,rotation=60, fontsize=9)

    ax.legend()

    fig.tight_layout()
    plt.savefig(device_name+'total.png',dpi=400)



def model_plot(arr,model):

    model_name=arr[0].split('/')[-1].split('_')[0]
    
    type=arr[0].split('/')[-1].split('_')[3]
    double=pd.read_csv(arr[0])
    half=pd.read_csv(arr[1])
    single=pd.read_csv(arr[2])
    if model.lower() =='densenet':
        n_groups = 4
        double=double[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        half=half[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        single=single[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
    elif model.lower() =='resnet':
        n_groups = 5
        double=double[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        half=half[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        single=single[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
    elif model.lower() =='squeezenet':
        n_groups = 2
        double=double[['squeezenet1_0','squeezenet1_1']]
        half=half[['squeezenet1_0','squeezenet1_1']]
        single=single[['squeezenet1_0','squeezenet1_1']]
    elif model.lower() =='vgg':
        n_groups = 4
        double=double[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        half=half[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        single=single[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
    else:
        raise NotImplementedError("To be implemented")


    means_double =double.mean().values
    std_double =double.std().values

    means_half =half.mean().values
    std_half =half.std().values

    means_single =single.mean().values
    std_single =single.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.25

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = ax.bar(index, means_double, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_double, error_kw=error_config,
                    label='double')

    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_half, error_kw=error_config,
                    label='half')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='g',
                    yerr=std_single, error_kw=error_config,
                    label='single')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title(model+'_'+type+"_"+model_name)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(double.columns,rotation=45, fontsize=9)

    ax.legend()
    fig.tight_layout()
    plt.savefig(model+'.png',dpi=300)


def arr_type(arr,type):
    arr=[x for x in arr if not '(2)' in x ]
    if type == 'single':
        temp=[x for x in arr if 'single' in x ]
    elif type == 'double':
        temp=[x for x in arr if 'double' in x ]
    elif type == 'half':
        temp=[x for x in arr if 'half' in x ]
    return temp

def model_plot2(arr,model):
    #model_name=arr[0].split('/')[-1].split('_')[0]
    type=arr[0].split('/')[-1].split('_')[3]
    ti1080=pd.read_csv(arr[0])
    ti2080=pd.read_csv(arr[1])
    titan=pd.read_csv(arr[2])
    if model.lower() =='densenet':
        n_groups = 4
        ti1080=ti1080[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        ti2080=ti2080[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
        titan=titan[['densenet121', 'densenet161', 'densenet169', 'densenet201']]
    elif model.lower() =='resnet':
        n_groups = 5
        ti1080=ti1080[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        ti2080=ti2080[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
        titan=titan[['resnet101','resnet152', 'resnet18', 'resnet34', 'resnet50']]
    elif model.lower() =='squeezenet':
        n_groups = 2
        ti1080=ti1080[['squeezenet1_0','squeezenet1_1']]
        ti2080=ti2080[['squeezenet1_0','squeezenet1_1']]
        titan=titan[['squeezenet1_0','squeezenet1_1']]
    elif model.lower() =='vgg':
        n_groups = 4
        ti1080=ti1080[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        ti2080=ti2080[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
        titan=titan[[ 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']]
    else:
        raise NotImplementedError("To be implemented")


    means_double =ti1080.mean().values
    std_double =ti1080.std().values

    means_half =ti2080.mean().values
    std_half =ti2080.std().values

    means_single =titan.mean().values
    std_single =titan.std().values

    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.25

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = ax.bar(index, means_double, bar_width,
                    alpha=opacity, color='b',
                    yerr=std_double, error_kw=error_config,
                    label='1080ti')

    rects2 = ax.bar(index + bar_width, means_half, bar_width,
                    alpha=opacity, color='r',
                    yerr=std_half, error_kw=error_config,
                    label='2080ti')

    rects2 = ax.bar(index + bar_width*2, means_single, bar_width,
                    alpha=opacity, color='g',
                    yerr=std_single, error_kw=error_config,
                    label='TitanV')

    ax.set_xlabel('models')
    ax.set_ylabel('times(ms)')
    ax.set_title(model+'_'+type)
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(ti1080.columns,rotation=45, fontsize=9)

    ax.legend()
    fig.tight_layout()
    plt.savefig(model+'_'+type+'_'+model_name+'.png',dpi=300)

In [1]:
type=arr[0].split('/')[-1].split('_')[3]

NameError: name 'arr' is not defined

In [8]:


#total_model(train,device_name)
total_model(inference,device_name)

C:\Users\Erik\Anaconda3\results\GeForce GTX 1060 6GB


In [9]:
from torchvision.models import resnet, densenet, vgg, squeezenet,inception
print(resnet.__all__)
print(squeezenet.__all__)
print(vgg.__all__)

['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
['SqueezeNet', 'squeezenet1_0', 'squeezenet1_1']
['VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19']
